In [13]:
import requests
import json
import csv
import os

class ProfessorNotFound(Exception):
    def __init__(self, search_argument, search_parameter="Name"):
        self.search_argument = search_argument
        self.search_parameter = search_parameter

    def __str__(self):
        return f"Professor not found: No match for {self.search_argument} under {self.search_parameter}."

class Professor:
    def __init__(self, ratemyprof_id, first_name, last_name, num_of_ratings, overall_rating):
        self.ratemyprof_id = ratemyprof_id
        self.first_name = first_name
        self.last_name = last_name
        self.num_of_ratings = num_of_ratings
        self.overall_rating = float(overall_rating) if num_of_ratings > 0 else 0.0

class RateMyProfApi:
    def __init__(self, school_id="1074"):
        self.UniversityId = school_id
        self.base_url = "https://www.ratemyprofessors.com/graphql"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
            "Content-Type": "application/json"
        }

        # Test Connection
        response = requests.post(self.base_url, headers=self.headers, json={})
        if response.status_code == 200:
            print("✅ Connected to RateMyProfessors API")
        else:
            print(f"❌ Connection Failed! Status Code: {response.status_code}")
            print("Response Content:", response.text)

        # try:
        #     self.professors = self.scrape_professors()
        # except Exception as e:
        #     print(f"❌ Error while scraping: {e}")

        # if not os.path.exists(f"SchoolID_{self.UniversityId}"):
        #     os.mkdir(f"SchoolID_{self.UniversityId}")

    # def scrape_professors(self):
    #     """Uses GraphQL API to fetch professors from the given university ID."""
    #     professors = {}

    #     query = """
    #     query GetProfessors($schoolId: ID!) {
    #         school(id: $schoolId) {
    #             professors(first: 100) {
    #                 edges {
    #                     node {
    #                         id
    #                         firstName
    #                         lastName
    #                         numRatings
    #                         avgRating
    #                     }
    #                 }
    #             }
    #         }
    #     }
    #     """
    #     payload = {
    #         "operationName": "GetProfessors",
    #         "variables": {"schoolId": str(self.UniversityId)},
    #         "query": query
    #     }
    #     print(payload)

    #     response = requests.post(self.base_url, headers=self.headers, json=payload)
    #     if response.status_code != 200:
    #         print(f"Error: Received status code {response.status_code}")
    #         return professors

    #     data = response.json()

    #     if "data" in data and data["data"]["school"] and "professors" in data["data"]["school"]:
    #         for prof in data["data"]["school"]["professors"]["edges"]:
    #             professor = prof["node"]
    #             prof_obj = Professor(
    #                 ratemyprof_id=professor["id"],
    #                 first_name=professor["firstName"],
    #                 last_name=professor["lastName"],
    #                 num_of_ratings=professor["numRatings"],
    #                 overall_rating=professor["avgRating"]
    #             )
    #             professors[prof_obj.ratemyprof_id] = prof_obj

    #     return professors



if __name__ == '__main__':
    school_id = "1530"  # Change this to your desired school ID
    rmp = RateMyProfApi(school_id)
    print(rmp.UniversityId)


✅ Connected to RateMyProfessors API
1530


In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Setup Chrome in headless mode
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open RMP and login manually
driver.get("https://www.ratemyprofessors.com/")

# Extract cookies after login
cookies = driver.get_cookies()
cookie_dict = {cookie["name"]: cookie["value"] for cookie in cookies}
print(cookie_dict)  # Copy this into your script


{'FCNEC': '%5B%5B%22AKsRol9DavoPdOO0MC9DgP7xVOg3SuesP0s8hh7FtRk2wpuPM6r7gNtIX2AaPL27345sgnk54_Q9VttyvY26bYPUNkIIiXKZQOIFvixWAd7ewzlxrpBOOGkJpG_pFHAh09SuBNhnO5WI8HE0KgjxG_Jtr-vtOEGvlw%3D%3D%22%5D%5D', '__eoi': 'ID=fefbf5b817d20a31:T=1740040673:RT=1740040673:S=AA-AfjYKTDMUNFM172Nso710nx2C', '_awl': '2.1740040673.5-f4157d65e59f8ecab482ce0be03f74c4-6763652d75732d7765737431-0', '__gads': 'ID=70f3ac6731e5ce41:T=1740040673:RT=1740040673:S=ALNI_MYdhAQZejWRNSnGfSkgVBYaYLxpTQ', '_au_1d': 'AU1D-0100-001740040674-G0O6ZBIY-UUM5', 'panoramaIdType': 'panoDevice', 'panoramaId_expiry': '1740645473272', '_cc_id': 'f6096d0fd68800d64900be52d4ee029a', 'lotame_domain_check': 'ratemyprofessors.com', 'panoramaId': '937179cb72ef948ed1e3966b76eb185ca02c4ebcd7e6ff0300f95bea922d7204', 'cto_bidid': 'frc9dF8lMkJBSUU1UllIQ2VGeTRiNlVUejdjU29rbFRlZ1FJcXRwUEI1aW8lMkJJSHJWV2tuR0VPMTR2MllFS0lMQmE5WlVlV2M1Y1VNTmRFNWFyRDBkakVSQVpZeXhwTHlrTTNvSkJyYmNqOGFWNWxqSDAlM0Q', 'pjs-unifiedid': '%7B%22TDID%22%3A%22c98be0ba-e38e-4053-

In [23]:
import requests

# GraphQL Query - Fetch Professors by School ID
query = """
query GetProfessors($schoolId: ID!) {
  school(id: $schoolId) {
    name
    professors(first: 10) {
      edges {
        node {
          id
          firstName
          lastName
          numRatings
          avgRating
        }
      }
    }
  }
}
"""

payload = {"query": query, "variables": {"schoolId": "1530"}}  # Example: University of Washington

# Headers with User-Agent and Referrer
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Content-Type": "application/json",
    "Referer": "https://www.ratemyprofessors.com/",
    "Origin": "https://www.ratemyprofessors.com",
    "Accept-Language": "en-US,en;q=0.9",
}

# Add Extracted Cookies
cookies = {
    "FCNEC": "%5B%5B%22AKsRol9DavoPdOO0MC9DgP7xVOg3SuesP0s8hh7FtRk2wpuPM6r7gNtIX2AaPL27345sgnk54_Q9VttyvY26bYPUNkIIiXKZQOIFvixWAd7ewzlxrpBOOGkJpG_pFHAh09SuBNhnO5WI8HE0KgjxG_Jtr-vtOEGvlw%3D%3D%22%5D%5D",
    "__eoi": "ID=fefbf5b817d20a31:T=1740040673:RT=1740040673:S=AA-AfjYKTDMUNFM172Nso710nx2C",
    "_awl": "2.1740040673.5-f4157d65e59f8ecab482ce0be03f74c4-6763652d75732d7765737431-0",
    "__gads": "ID=70f3ac6731e5ce41:T=1740040673:RT=1740040673:S=ALNI_MYdhAQZejWRNSnGfSkgVBYaYLxpTQ",
    "_au_1d": "AU1D-0100-001740040674-G0O6ZBIY-UUM5",
    "panoramaIdType": "panoDevice",
    "panoramaId_expiry": "1740645473272",
    "_cc_id": "f6096d0fd68800d64900be52d4ee029a",
    "lotame_domain_check": "ratemyprofessors.com",
    "panoramaId": "937179cb72ef948ed1e3966b76eb185ca02c4ebcd7e6ff0300f95bea922d7204",
    "cto_bidid": "frc9dF8lMkJBSUU1UllIQ2VGeTRiNlVUejdjU29rbFRlZ1FJcXRwUEI1aW8lMkJJSHJWV2tuR0VPMTR2MllFS0lMQmE5WlVlV2M1Y1VNTmRFNWFyRDBkakVSQVpZeXhwTHlrTTNvSkJyYmNqOGFWNWxqSDAlM0Q",
    "pjs-unifiedid": "%7B%22TDID%22%3A%22c98be0ba-e38e-4053-b4c8-6d56d90e90bb%22%2C%22TDID_LOOKUP%22%3A%22FALSE%22%2C%22TDID_CREATED_AT%22%3A%222025-02-20T08%3A37%3A52%22%7D",
    "_ga": "GA1.1.2133600686.1740040673",
    "_ga_WET17VWCJ3": "GS1.1.1740040672.1.0.1740040672.0.0.0",
    "pjs-unifiedid_cst": "zix7LPQsHA%3D%3D",
    "_pubcid_cst": "zix7LPQsHA%3D%3D",
    "__gpi": "UID=0000105399ae7752:T=1740040673:RT=1740040673:S=ALNI_MbpLX_nyFFU7Ld9JozCjQUZOVb3Yg",
    "cto_bundle": "-ruGBl83M3JqTWhReEZHejVwZXhkNXBWald6cFo5Z1VncDBsZzMyc0lKeWNaT2dNcDUzckhaeVlaayUyQk9JUFpYUHdzJTJGcFppTWZnYVRRclMydVZGeSUyQk52QnhxN2dXck1ZaERpSjRITkowTjc5UHE3eDZCSU5FMlc0UXh0aU42QjZQJTJGcW4w",
    "_pubcid": "0c8cb645-e038-4f38-a398-43fd693be4bc",
    "RMP_AUTH_COOKIE_VERSION": "v01"
}

# Create a session for persistent cookies
session = requests.Session()
session.headers.update(headers)
session.cookies.update(cookies)

# Send Request
response = session.post("https://www.ratemyprofessors.com/graphql", json=payload)

# Print Response
print(f"Status Code: {response.status_code}")
print(f"Response Headers: {response.headers}")
print(f"Response Text:\n{response.text[:1000]}")  # Print first 1000 characters for verification


Status Code: 200
Response Headers: {'Date': 'Thu, 20 Feb 2025 08:39:17 GMT', 'Content-Type': 'text/plain; charset=utf-8', 'Content-Length': '13', 'Connection': 'keep-alive', 'Server': 'nginx/1.22.0', 'Vary': 'Origin', 'X-Content-Type-Options': 'nosniff'}
Response Text:
Unauthorized



In [26]:
import ratemyprofessor as rmp
SCHOOL = rmp.School(school_id=1257)
print(SCHOOL)

In [ ]:
import ratemyprofessor as rmp

# Tries to find the University of Minnesota - Twin Cities on RateMyProfessor.
SCHOOL = rmp.School(school_id=1530)

def get_professors_by_name(name):
    try:
        professors = rmp.get_professors_by_school_and_name(SCHOOL, name)
        if not professors:
            print(f"No matches found for {name}")
        else:
            for prof in professors:

                print(f"Found: {prof.name}, Rating: {prof.rating}, Difficulty: {prof.difficulty}, Link: https://www.ratemyprofessors.com/professor/{prof.id}")
    except Exception as e:
        print(f"Error retrieving {name}: {e}")

if __name__ == "__main__":
    test_names = ["Keila Denise Taylor"]  # Replace with test professor names
    for name in test_names:
        get_professors_by_name(name)


Found: Keila Taylor, Rating: 4.3, Difficulty: 3, Link: https://www.ratemyprofessors.com/professor/2506375
Found: Denise Grollmus, Rating: 5, Difficulty: 2.6, Link: https://www.ratemyprofessors.com/professor/2023506
Found: Denise Anderson, Rating: 3.8, Difficulty: 3.7, Link: https://www.ratemyprofessors.com/professor/849505
Found: Denise Wilson, Rating: 2.4, Difficulty: 4.3, Link: https://www.ratemyprofessors.com/professor/509425
Found: Jesse Taylor, Rating: 2.8, Difficulty: 3.6, Link: https://www.ratemyprofessors.com/professor/1989486
Found: Taylor Soja, Rating: 5, Difficulty: 3, Link: https://www.ratemyprofessors.com/professor/2714261
Found: Maria Taylor, Rating: 3.5, Difficulty: 3.5, Link: https://www.ratemyprofessors.com/professor/2571714
Found: Alyssa Taylor, Rating: 4.4, Difficulty: 1.9, Link: https://www.ratemyprofessors.com/professor/1749861


TypeError: 'NoneType' object is not subscriptable